In [ ]:
%load_ext autoreload
%autoreload 2

from modules.auth import *
from modules.assessments_endpoints import *
from modules.frame_transformations import *
from modules.config import base_url_illuminate
import logging
import os

os.makedirs('logs', exist_ok=True)
logging.basicConfig(filename='./logs/Illuminate_operations.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)
logging.info('\n\n-------------New Illuminate Operations Logging Instance')

def log_start():
    logging.info('\n\n-------------New Illuminate Operations Logging Instance')

def get_assessment_results(save_path, view_path, years_data, start_date, end_date_override=None):
    log_start()
    try:
        access_token, expires_in = get_access_token()

        assessments_df, assessment_id_list = get_all_assessments_metadata(access_token)
        # assessment_id_list = assessment_id_list[:10] #for testing
        missing_ids_from_metadata = ['114845', '141498'] # Add assessments that are not present in assessements metadata
        assessment_id_list = list(set(assessment_id_list + missing_ids_from_metadata))
        logging.info(f'Here is the length of the assessment_id_list variable {len(assessment_id_list)}')
        
        test_results_group, log_results_group = loop_through_assessment_scores(access_token, assessment_id_list, 'Group', start_date, end_date_override)
        test_results_standard, log_results_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'Standard', start_date, end_date_override)
        test_results_no_standard, log_results_no_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'No_Standard', start_date, end_date_override)
 
        test_results_combined = bring_together_test_results(test_results_no_standard, test_results_standard)
        test_results_view = create_test_results_view(test_results_combined, years_data) #add in grade level col, string matching
        logging.info("Assessment results fetched and processed.")

        
        os.makedirs(save_path, exist_ok=True)

        if years_data == '23-24':
            logging.info(f'Sending data for {years_data} school year')
            send_to_local(save_path, test_results_group, 'assessment_results_group_historical.csv')
            send_to_local(save_path, test_results_combined, 'assessment_results_combined_historical.csv')
            send_to_local(view_path, test_results_view, 'illuminate_assessment_results_historical.csv')
            
        elif years_data == '24-25':
            logging.info(f'Sending data for {years_data} school year')
            send_to_local(save_path, test_results_group, 'assessment_results_group.csv')
            send_to_local(save_path, test_results_combined, 'assessment_results_combined.csv')
            send_to_local(view_path, test_results_view, 'illuminate_assessment_results.csv')
        else:
            raise ValueError(f'Unexpected value for years variable data {years_data}')
        
        return(test_results_group, test_results_view, test_results_combined)


    except Exception as e:
        logging.error(f"Error fetching assessment results: {e}")
        raise AirflowException("Failed to fetch and process assessment results")


test_results_group, test_results_view, test_results_combined = get_assessment_results(save_path = '/home/g2015samtaylor/illuminate',
                                                                                      view_path = '/home/g2015samtaylor/views',
                                                                                      years_data = '23-24',
                                                                                      start_date = '2023-07-01',
                                                                                      end_date_override='2024-07-01') #If not specified it will default to none aka todays date



#Make this to where you can easily pull in historical. Takes a lot of time to go back and get histroical with manually altering the API strings.
#Add in assessments_metadata as a consistent table

#Actions steps

#Finish packaging year parameter. 
#Update changes in docker file
#Make sure changes flow through to airflow
#Run locally for string matching


In [203]:
# Hey Jenny, 
# So it looks like 132 of the titles coming from results-20250106-08115 excel file are missing in the Illuminate assessments endpoint. 
# I have went back and made sure the code logic is flowing and everything available is being grabbed from the Illuminate assessments endpoint.
# Without any matching title, or matching assessment_id I am unsure of what to do with these moving forward. 
# A couple of questions for you is where did you get these title names from, and can you obtain the assessment ids for these?



In [155]:

import pandas as pd
from modules.frame_transformations import *
pd.set_option('display.max_colwidth', None)

fixes = pd.read_csv('/home/g2015samtaylor/airflow/git_directory/Illuminate/illuminate_historical_column_fixes_2324.csv')
v = pd.read_csv('/home/g2015samtaylor/views/illuminate_assessment_results_historical.csv') 

#Birng back jennys produced excel frame with the master assessments frame
access_token, expires_in = get_access_token()
temp, assessments_df = merge_excel_with_assessments_master_on_title(access_token)

#Right only are titles that are strictly in the produced excel file
#Present means the titles are present in teh excel file and the assessments endpoint
missing = temp.loc[temp['_merge'] == 'right_only']
present = temp.loc[temp['_merge'] == 'both']


In [183]:
display(present[['title', 'current grade', 'updated grade', 'curriculum', 'updated curriculum']].tail(4)) # 130 total titles present

print('Here is the view below with the title present')
v.loc[v['title'].str.contains('Kinder - IM Unit 5 Checkpoint A',case=False)][['title', 'grade', 'curriculum']].drop_duplicates()

,title,current grade,updated grade,curriculum,updated curriculum
274,Kinder - IM Unit 5 Checkpoint A,0.0,0,Checkpoint,Checkpoint
281,Kinder - IM Unit 6 Checkpoint A,1.0,0,Checkpoint,Checkpoint
282,Kinder - IM Unit 6 Checkpoint A,0.0,0,Checkpoint,Checkpoint
285,Kinder - IM Unit 6 Checkpoint C,1.0,0,Checkpoint,Checkpoint


Here is the view below with the titel present


,title,grade,curriculum
98786,Kinder - IM Unit 5 Checkpoint A,1.0,Checkpoint
98886,Kinder - IM Unit 5 Checkpoint A,0.0,Checkpoint


In [196]:
display(missing[['title']])

print('Teseting if partial string is present')
assessments_df.loc[assessments_df['title'].str.contains('1st Grade IM Unit', case=False)]['title']

#Looks like 132 of the titles are completely missing from Illuminate when I pull from their assessments endpoint. 
#These are titles that originagte from the results-20250106-085115 excel file. 

#Without any matching title, or matching assessment_id I am unsure of what to do with these moving forward. 

# A couple of questions is where did you get these title names from? Can you obtain the assessments ids for these if so?

#The strings matching or assessment_id matching is possible when the titles are present

,title
0,1st Grade - IM Unit 1 - End of Unit Assessment
1,1st Grade - IM Unit 2 - End of Unit Assessment
3,1st Grade - IM Unit 3 - End of Unit Assessment
4,1st Grade - IM Unit 3 - End of Unit Assessment
9,1st Grade - IM Unit 4 - End of Unit Assessment
...,...
287,Kinder - IM Unit 7 - End of Unit Assessment
288,Kinder - IM Unit 7 Checkpoint A
289,Kinder - IM Unit 7 Checkpoint A
290,Kinder - IM Unit 7 Checkpoint B


Teseting if partial string is present


Series([], Name: title, dtype: object)

In [197]:
assessments_df.loc[assessments_df['title'].str.contains('Kinder - IM Unit 7', case=False)]
#Missing Kinder - IM Unit 6 Checkpoint B

,assessment_id,local_assessment_id,title,description,author_id,district_author_id,author_first,author_last,created_at,updated_at,scope_id,scope,subject_id,subject,grade_levels,__count


In [198]:
assessments_df.loc[assessments_df['title'].str.contains('Kinder - IM Unit 8', case=False)]
#Missing edn of unit assessment

,assessment_id,local_assessment_id,title,description,author_id,district_author_id,author_first,author_last,created_at,updated_at,scope_id,scope,subject_id,subject,grade_levels,__count
954,107742,None,Kinder - IM Unit 8 Checkpoint A,Q1-Q3 C=correct I=incorrect,3057,3057,Andrew,Short,2023-07-31 15:03:25,2023-07-31 15:06:33,11,Grade Level,2,Mathematics,K,1126
955,107743,None,Kinder - IM Unit 8 Checkpoint B,Q1- Q2 C=correct I=incorrect,3057,3057,Andrew,Short,2023-07-31 15:09:18,2023-07-31 15:11:41,11,Grade Level,2,Mathematics,K,1126
956,107744,None,Kinder - IM Unit 8 Checkpoint C,Q1-Q3 C=correct I=incorrect,3057,3057,Andrew,Short,2023-07-31 15:14:17,2023-07-31 15:17:12,11,Grade Level,2,Mathematics,K,1126
957,107745,None,Kinder - IM Unit 8 Checkpoint D,Q1-Q4 C=correct I=incorrect,3057,3057,Andrew,Short,2023-07-31 15:19:11,2023-07-31 15:22:31,None,None,2,Mathematics,None,1126


# Re-testing and double checking get_all_assessments_metadata()

In [136]:
# https://demo.illuminateed.com/live/rest_server.php/Api/Assessments/

# Set the initial page and an empty DataFrame to store all results
page = 1
all_results = pd.DataFrame()

#To ensure all pages are looped through properly
# while True:

#Base URL and headers for API requests
url_ext = f'Assessments/?page={page}&limit=1000'
headers = {"Authorization": f"Bearer {access_token}"}

logging.info(f'Fetching data from {base_url_illuminate + url_ext}')

# try:
    # Make the API request with the current page number
response = requests.get(base_url_illuminate + url_ext.format(url_ext), headers=headers)

#From raw request here are the outputs
# 'page': 1,
#  'num_pages': 2,
#  'num_results': 1126,

In [ ]:
access_token, expires_in = get_access_token()
assessments_df, assessment_id_list = get_all_assessments_metadata(access_token)

#All possible assessments from this endpoint are grabbed
assessments_df['assessment_id'].nunique() == 1126

True

# Efforts


#Based on complete string matching of the title column between results-20250106-085115 excel file & all of the assessments found at the  in the views db.

#When merging with the illuminate_assessments_results there are 160 titles that are present in the illuminate_assessments_results table & 132 that are not are not found.

#Becuase these are strictly in the produced excel file, hence there is string matching that can occur on the title.

------------------------------------------
# Example - because titles with IM has multiple grades I can not apply a singule updated grade to all based on IM being in the title
# However it will work for updated curriculum. 
